# Abstract
This notebook shows ho to generate a composite graph, out of many subgraphs requested in succession to a Neo4j DBMS.

The method can be useful whenever the user must generate a single graph, limiting the connected nodes with complex limitations, using functionalities currently not supported by the cypher language.

The context is that of a web application, where a server component performs the extractions from Neo4J and transforms or aggregated the extracted graphs, before sending the results to the client for visualisation or further analysis.

As an example, consider extracting top N elements from N different types of nodes, connected to a determined node.

In the context of parliamentary activity, considering a given Issue, and all connected Legislative Acts, Meetings, and Stakeholders, if whe have these relations:
- (Act)-[:IS_ABOUT]->(Issue)
- (Meeting)-[:IS_ABOUT]->(Issue)
- (Stakeholder)-[:FOLLOWS]->(Issue)

then trying to extract these nodes:
- last 10 Acts, ordered by last activity date,
- last 10 Meetings
- top 20 relevant Stakeholders

is a complex task, for a single cypher expression.

Subdividing the task into 4 different simple extractions, and merging the resulting graphs, helps obtaining the desired result.

# Architecture
[py2neo](http://py2neo.org/v3/index.html) will be used, as it allows working with `py2neo.Subgraph`s.

The connection to the Neo4J instance is made to the localhost, through the _bolt_ protocol, using authentication.

In [1]:
from py2neo import Graph, Subgraph
from py2neo import types

In [2]:
graph = Graph("bolt://localhost", username="neo4j", password="admin")

Graphs are extracted through a call to `py2neo.graph.data`, that allows to work easily with the results using flexible data structures.

The first extraction gets the Issues (Macro and Micro), connected to a given Customer.

In [3]:
results = graph.data(
"""
match (c:Cliente {id_cliente: 59})
match (tc:TemaMicro {id_tema: 169})
match (c)-[c_tc:SI_OCCUPA]->(tc)-[t_tc:FIGLIO]->(t:TemaMacro)
return c, tc, c_tc, t_tc
"""
)

The following snippet will build the `nodes` and `rels` lists, which are going to be used to build a `py2neo.Subgraph`.

In [4]:
nodes = []
rels = []
for rec in results:
    for k, item in rec.items():
        if type(item) == types.Node:
            nodes.append(item)
        elif type(item) == types.Relationship:
            rels.append(item)
        else:
            print(type(item))
            
jti_issue_169 = Subgraph(nodes, rels)

The same operations are then performed to fetch connected Acts, limiting the result to the last 10, according to a given property wishin the nodes, and leaving the rest of the graph out (only tc is returned).

In [5]:
results = graph.data(
"""
match (c:Cliente {id_cliente: 59})
match (tc:TemaMicro {id_tema: 169})
match (c)-[c_tc:SI_OCCUPA]->(tc)<-[tc_pr:RIGUARDA]-(pr:Provvedimento)
with tc, pr, tc_pr order by pr.data_ultimo_rapporto desc limit 10
return tc, pr, tc_pr
""")

nodes = []
rels = []
for rec in results:
    for k, item in rec.items():
        if type(item) == types.Node:
            nodes.append(item)
        elif type(item) == types.Relationship:
            rels.append(item)
        else:
            print(type(item))
            
jti_top10_provv_issue_169 = Subgraph(nodes, rels)

The 2 subgraphs can now be **merged** using the set operator `|`.

In [6]:
jti = jti_issue_169 | jti_top10_provv_issue_169

The resulting graph can easily be analyzed and transformed.

In [7]:
n0 = list(jti.nodes())[0]

In [8]:
dict(n0)

{'alias': 'DDL Consenso informato',
 'aula': 'camera',
 'data_presentazione': '2013-07-29',
 'data_ultimo_report': '2017-01-10',
 'firmatari': ['On.Delia Murer'],
 'id_provvedimento': 3742,
 'iniziativa': 'Parlamentare',
 'lettura': '',
 'name': 'DDL Consenso informato',
 'numero': 'AC 1432',
 'relevance': 0.34741476799062865,
 'sede': '',
 'status': '',
 'titolo': ''}

In [9]:
r0 = list(jti.relationships())[0]

In [10]:
dict(r0)

{'clienti': []}

# Function
The following function returns a `py2neo.subgraph`, starting from a cypher query.

Due to internal workings of the `Subgraph` class the query must return at least one node, 
or an exception will be raised.

In [11]:
def get_subgraph(query, params=None):
    """ Estrae un `py2neo.Subgraph` a partire da una query cypher.
    Può essere usato per effettuare dei merge e generare dei grafi compositi.

    E' necessario che sia presenta almeno un nodo, altrimenti la funzione ritorna
    un errore.

    :param query:  La query cypher
    :param params: I parametri della query, se necessari
    :return:       Il subgrafo
    :rtype:        `py2neo.Subgraph`
    """
    results = graph.data(query, params)

    nodes = []
    rels = []
    for rec in results:
        for k, item in rec.items():
            if type(item) == types.Node:
                nodes.append(item)
            elif type(item) == types.Relationship:
                rels.append(item)
    if nodes:
        return Subgraph(nodes, rels)    
    else:
        raise Exception("At least one node is required")

In [12]:
cid = 59
tid = 169
microtema = get_subgraph("""
    match (c:Cliente)
    match (tc:TemaMicro)
    where c.id_cliente={cid} and tc.id_tema={tid}
    match (c)-[c_tc:SI_OCCUPA]->(tc)-[t_tc:FIGLIO]->(t:TemaMacro)
    return c, tc, c_tc, t_tc, t
""", params={'cid': cid, 'tid': tid}
)
provv = get_subgraph("""
    match (c:Cliente)
    match (tc:TemaMicro)
    where c.id_cliente={cid} and tc.id_tema={tid}
    match (c)-[c_tc:SI_OCCUPA]->(tc)<-[tc_pr:RIGUARDA]-(pr:Provvedimento)
    with tc, pr, tc_pr order by pr.data_ultimo_rapporto desc limit 10
    return tc, pr, tc_pr
""", params={'cid': cid, 'tid': tid}
)
incontri = get_subgraph("""
    match (c:Cliente)
    match (tc:TemaMicro)
    where c.id_cliente={cid} and tc.id_tema={tid}
    match (c)-[c_tc:SI_OCCUPA]->(tc)<-[tc_i:RIGUARDA]-(i:Incontro)
    with tc, i, tc_i order by i.data desc limit 10
    return tc, i, tc_i
""", params={'cid': cid, 'tid': tid}
)
persone = get_subgraph("""
    match (c:Cliente)
    match (tc:TemaMicro)
    where c.id_cliente={cid} and tc.id_tema={tid}
    match (c)-[c_tc:SI_OCCUPA]->(tc)-[t_tc:FIGLIO]->(t:TemaMacro)
    optional match (tc)<-[tc_p:SI_OCCUPA]-(p:Persona)
    return t, t_tc, tc, tc_p, p order by p.relevance desc limit 20
""", params={'cid': cid, 'tid': tid}
)

In [13]:
composite = microtema | provv | incontri | persone

In [14]:
list(composite.relationships())

[(ff8f026)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (b37cf42)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (e6d0fb2)-[:RIGUARDA]->(d31220f),
 (bb506e0)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (c4d4188)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (a613bb0)-[:RIGUARDA]->(d31220f),
 (c38385e)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (cbc2439)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (e8843bf)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (c348a5d)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (b095c4d)-[:RIGUARDA]->(d31220f),
 (c9f03e9)-[:RIGUARDA]->(d31220f),
 (ec54803)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (ffa77eb)-[:RIGUARDA]->(d31220f),
 (b4c057c)-[:RIGUARDA]->(d31220f),
 (d31220f)-[:FIGLIO]->(c365ea4),
 (cfd2b19)-[:SI_OCCUPA {valore:"0"}]->(d31220f),
 (aaa54cc)-[:RIGUARDA {clienti:[]}]->(d31220f),
 (ceb5de1)-[:RIGUARDA]->(d31220f),
 (fe09e46)-[:RIGUARDA]->(d31220f),
 (c777551)-[:RIGUARDA]->(d31220f),
 (bd4f895)-[:RIGUARDA]->(d31220f)]